# LangChain Expression Language

In [1]:
%pip install langchain langchain_openai --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 399.9/399.9 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.2/292.2 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.0/383.0 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 35.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.7 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existin

In [2]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

··········


In [3]:
from operator import itemgetter
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableBranch, RunnablePassthrough

------------------------------------------

## Branching Out with _Conditional Logic:_
The runnable is initialized with a list of (condition, runnable) pairs and a default branch.

When operating on an input, the first condition that evaluates to True is selected, and the corresponding runnable is run on the input.

If no condition evaluates to True, the default branch is run on the input.

In [4]:
branch = RunnableBranch(
                (lambda x: x == 'hello', lambda x: x),
                (lambda x: isinstance(x, str), lambda x: x.upper()),
                (lambda x: "This is the default case, in case no above lambda functions match."),
            )

print(branch.invoke("hello")) # "hello"
print(branch.invoke(None)) # "This is the default case"

hello
This is the default case, in case no above lambda functions match.


---

## Branching and _Merging_

```
     Input
      / \
     /   \
 Branch1 Branch2
     \   /
      \ /
    Combine

```

### First Example:

In [5]:
planner = (
    ChatPromptTemplate.from_template("Generate an argument about: {input}")
    | ChatOpenAI()
    | StrOutputParser()
    | {"base_response": RunnablePassthrough()}
)

arguments_for = (
    ChatPromptTemplate.from_template(
        "List the pros or positive aspects of {base_response}"
    )
    | ChatOpenAI()
    | StrOutputParser()
)

arguments_against = (
    ChatPromptTemplate.from_template(
        "List the cons or negative aspects of {base_response}"
    )
    | ChatOpenAI()
    | StrOutputParser()
)

final_responder = (
    ChatPromptTemplate.from_messages(
        [
            ("ai", "{original_response}"),
            ("human", "Pros:\n{results_1}\n\nCons:\n{results_2}"),
            ("system", "Generate a final response given the critique"),
        ]
    )
    | ChatOpenAI()
    | StrOutputParser()
)

chain = (
    planner
    | {
        "results_1": arguments_for,
        "results_2": arguments_against,
        "original_response": itemgetter("base_response"),
    }
    | final_responder
)

In [6]:
chain.invoke({"input": "scrum"})

'While scrum offers numerous benefits such as promoting collaboration, increasing productivity, and providing a structured framework for project management, it is important to consider the potential drawbacks associated with this methodology.\n\nThe lack of predictability in project outcomes, high level of commitment required from team members, dependency on team dynamics, potential for scope creep, and lack of documentation are valid concerns that can impact the success of scrum implementations. It is essential for teams considering scrum to carefully assess their project requirements, team dynamics, and organizational culture to determine if scrum is the right fit for their specific needs.\n\nBy addressing these challenges through proper training, communication, and project management practices, teams can mitigate the risks associated with scrum and leverage its benefits to deliver high-quality results in a collaborative and adaptive manner. Ultimately, the key is to strike a balance

### Example Two:

In [7]:
joke_chain = (
    ChatPromptTemplate.from_template("Tell me a joke about {topic}")
    | ChatOpenAI()
    | StrOutputParser()
    | {"joke": RunnablePassthrough(), "topic": RunnablePassthrough()}
)

explain_joke = (
    ChatPromptTemplate.from_template("Explain the joke: {joke}")
    | ChatOpenAI()
    | StrOutputParser()
)

benefits_of_joke = (
    ChatPromptTemplate.from_template("List the benefits of this joke: {joke}")
    | ChatOpenAI()
    | StrOutputParser()
)

final_responder = (
    ChatPromptTemplate.from_messages(
        [
            ("system", "You are responsible for generating a small analysis of a joke. The topic will be: {topic}"),
            ("ai", "{joke}. The benefits of this joke are: {benefits}"),
            ("human", "The explanation of the joke is: {explanation}"),
            ("human", "Generate a small analysis of the joke. Analysis: "),
        ]
    )
    | ChatOpenAI()
    | StrOutputParser()
)

final_chain = (
    {"topic": RunnablePassthrough()}
    | joke_chain
    | {
        "explanation": explain_joke,
        "benefits": benefits_of_joke,
        "joke": itemgetter("joke"),
        "topic": itemgetter("topic"),
    }
    | final_responder
)

final_chain.invoke({"topic": "bears"})

'The joke "Why did the bear break up with his girlfriend? Because he couldn\'t bear the relationship any longer!" utilizes a clever play on words to create humor. By using the dual meaning of the word "bear" (as the animal and as a verb meaning to endure), the joke sets up an unexpected punchline that plays on the idea of a bear experiencing relationship issues. This unexpected twist adds an element of surprise and silliness to the joke, making it entertaining for the audience. Additionally, the joke\'s simplicity and relatability make it easy for a wide range of people to understand and appreciate, enhancing its effectiveness in eliciting laughter and amusement.'